## This notebook is adapted from Yueheng's Mathematica Notebook. 

This notebook requires installation of the astropy, numpy package to keep track of units. Please download the package using conda or your preferred package management software. 

In [1]:
import numpy as np
import astropy.units as u
import astropy.constants as c

In [2]:
phi0 = c.h/2/c.e.si
T = 30*u.mK
epsilon_r = 11.7


### Define some Transmon parameters and calculate transmon only numbers

In [3]:
Cq = 110*u.fF
Lj = 12*u.nH
Ec = (c.e.si**2/2/Cq).to(u.J)
Ej = ((phi0/2/np.pi)**2/Lj).to(u.J)
# epsilon1 = -Ec*2**9
wq = (np.sqrt(8*Ej*Ec)-Ec)/c.hbar
fq = wq/2/np.pi
alpha = Ec/c.h

In [4]:
print('Transmon Frequency is:',fq.to(u.GHz))
print('The anharmonicity is:', alpha.to(u.MHz))
print('The ratio  Ej/Ec is:', Ej/Ec)

Transmon Frequency is: 4.2045026407625805 GHz
The anharmonicity is: 176.09299386053743 MHz
The ratio  Ej/Ec is: 77.35567687994813


### Define some Resonator parameters and resonator only numbers

In [5]:
fr = 7.5*u.GHz
wr = fr*2*np.pi
lam  = c.c*(1/fr).to(u.s)#why is this the speed of light?

#Capacitance and Inductance per unit length calculated using 
# https://wcalc.sourceforge.net/cgi-bin/coplanar.cgi?wc_unit_menu_1_0=nH&wc_unit_menu_1_1=m&wc_unit_menu_2_0=mOhm&wc_unit_menu_2_1=m&wc_unit_menu_3_0=fF&wc_unit_menu_3_1=m&wc_unit_menu_4_0=uMho&wc_unit_menu_4_1=inch&wc_unit_menu_7_0=dB&wc_unit_menu_8_0=dB&wc_unit_menu_8_1=inch&wc_unit_menu_12_0=mil&wc_unit_menu_11_0=ns&wc_unit_menu_13_0=inch&w=13&wc_unit_menu_0_0=um&s=9&l=4&wc_unit_menu_5_0=mm&tmet=0.25&rho=0&wc_unit_menu_9_0=Ohm&wc_unit_menu_9_1=m&rough=0&wc_unit_menu_10_0=mil&h=350&es=11.45&tand=0&freq=7&wc_unit_menu_6_0=GHz&analyze=Analyze&Ro=51.7611&elen=84.3896
cc = 165*u.pF/u.m
ll = 423*u.nH/u.m
#the above capacitance and indictance gives the speed of light in the transmisison line
v = 1/np.sqrt(ll*cc).si

L= lam/4
# Cr = cc*L
Cr = 1600*u.fF
# v0 = 

lr = (1/Cr/wr**2).to(u.nH)
delta = wq-wr
delta_f = delta/2/np.pi

n = (np.exp(c.hbar*wr/c.k_B/T)-1)**(-1)#probability of particle in the excited state of the transmon

In [6]:
print('Speed of light i the resonator is:', v)
v/c.c

Speed of light i the resonator is: 119698261.90109022 m / s


<Quantity 0.39927042>

In [7]:
print('The detuning between transmon and resonator is:', delta_f.to(u.GHz))
print('The probability of states in the excited state is:', n)

The detuning between transmon and resonator is: -3.2954973592374195 GHz
The probability of states in the excited state is: 6.155888050717911e-06


### Qubit-Resonator Coupling Parameters

In [8]:
Cg = 5*u.fF
Cj = 0*u.fF
C_sum = Cq+Cj


Zvac = np.sqrt(c.mu0/c.eps0).to(u.Ohm)
Zr = np.sqrt(lr/Cr).to(u.Ohm)
alpha_fine = 1/4/np.pi/c.eps0*c.e.si**2/c.hbar/c.c
Rk = c.h/c.e.si**2

g_1 = wr*Cg/C_sum*(Ej/2/Ec)**(1/4)*np.sqrt(Zr/Zvac)*np.sqrt(2*np.pi*alpha_fine)
g_2 = wr*Cg/C_sum*(Ej/2/Ec)**(1/4)*np.sqrt(np.pi*Zr/Rk)
chi = -g_1**2*Ec/c.hbar/delta/(delta-Ec/c.hbar)
wr_coupled = wr-g_1**2/(delta-Ec/c.hbar)
wq_coupled = wq+g_1**2/delta
kappa = 2*chi.decompose()

In [9]:
print('The coupling strength between the resonator and the transmon (g) is:', g_1.to(u.MHz)/2/np.pi)
print('The dispersive shift of the cavity is:', chi.to(u.MHz))
print('The qubit frequency changed from ', wq.to(u.GHz)/2/np.pi, ' to ', wq_coupled.to(u.GHz)/2/np.pi)
print('The cavity frequency changed from ', wr.to(u.GHz)/2/np.pi, ' to ', wr_coupled.to(u.GHz)/2/np.pi)

The coupling strength between the resonator and the transmon (g) is: 34.157092000597636 MHz
The dispersive shift of the cavity is: -0.1128325189391839 MHz
The qubit frequency changed from  4.2045026407625805 GHz  to  4.204148610154524 GHz
The cavity frequency changed from  7.499999999999999 GHz  to  7.500336072754925 GHz


In [11]:
kappa.to(u.MHz)

<Quantity -0.22566504 MHz>

In [10]:
n_crit = 1/3*(np.absolute(delta-Ec/c.hbar)**2/4/g_1**2-1)

In [11]:
print('A conservative estimate of critical photon numbers in the system gives:', n_crit)

A conservative estimate of critical photon numbers in the system gives: 777.0205241094756


### Qubit coherence parameters

In [12]:
def lambda_func(omega):
    return(Ck/np.sqrt(c.c*Cr))*np.sqrt(omega*wr/2/np.pi/v)

In [13]:
Ck = 6*u.fF #coupling between the transmission line and the resonator

#decay due to spontaneous emission due to coupling
gamma = (g_1**2*np.absolute(kappa)/delta**2).si #formula 3.55 from yale thesis
T1 = 1/gamma

#dephasing due to charge noise
A = 1e-4
epsilon_1 = (-1)*Ec*2**(4+5)/1*np.sqrt(2/np.pi)*(Ej/2/Ec)**(1/2+3/4)*np.exp(-np.sqrt(8*Ej/Ec))
Tphi_charge = c.hbar/A/np.pi/np.absolute(epsilon_1)

#dephasing due to flux noise
A = 1e-5*phi0
Tphi_flux = c.hbar*phi0**2/(A**2*np.pi**2*np.sqrt(2*Ej*Ec))

In [14]:
print('The relaxation time due to spontaneous emission is:', T1.to(u.us))
print('The dephasing time due to charge noise is:', Tphi_charge.to(u.ms))
print('The dephasing time due to flux noise is:', Tphi_flux.to(u.ms))

The relaxation time due to spontaneous emission is: 30094.622538819087 us
The dephasing time due to charge noise is: 651.7250020006401 ms
The dephasing time due to flux noise is: 73.06374458263042 ms
